<a href="https://colab.research.google.com/github/Ruthuvikas/Stress-tweet-detection/blob/main/Stress_tweet_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/Xdata/archive.zip

Archive:  /content/drive/MyDrive/Xdata/archive.zip
  inflating: Reddit_Combi.csv        
  inflating: Reddit_Title.csv        
  inflating: Twitter_ Non-Advert-Tabelle 1.csv  
  inflating: Twitter_Full.csv        


In [ ]:
import pandas as pd

USECOLS = ['text', 'labels']
df = pd.read_csv(filepath_or_buffer='Twitter_Full.csv', sep=';', usecols=USECOLS)
df.head()


,text,labels
0,Being s mom is cleaning 24/7 the same shit ove...,1
1,And now we have been given the walkthru book b...,0
2,Wishing YOU Peace Joy & Love! JoyTrain MentalH...,0
3,speak-no-evil monkey Can I Be Honest With You...,1
4,Psy Do u hv any regrets? Me No Psy Are you hap...,0


In [ ]:
df.columns

Index(['text', 'labels'], dtype='object')

In [ ]:
df

,text,labels
0,Being s mom is cleaning 24/7 the same shit ove...,1
1,And now we have been given the walkthru book b...,0
2,Wishing YOU Peace Joy & Love! JoyTrain MentalH...,0
3,speak-no-evil monkey Can I Be Honest With You...,1
4,Psy Do u hv any regrets? Me No Psy Are you hap...,0
...,...,...
8895,Thank you for all your business questions toda...,0
8896,Who is planning on walking their minds today?!...,1
8897,no mobile phones Three Tips to Reduce Technos...,1
8898,purple heart Day 1 purple heart But vets firs...,1


In [ ]:
df['text'][0]

'Being s mom is cleaning 24/7 the same shit over and over with the end result with everything being a mess again. momlife kids tired'

In [ ]:
df['text'].fillna("", inplace=True)

In [ ]:
import pandas as pd
import tensorflow_hub as hub
from tqdm import tqdm

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to convert tweets to vectors
def tweet_to_vector(tweet):
    return embed([tweet])[0].numpy()

# Convert each tweet to a vector and store the result
vectors = []
for tweet in tqdm(df['text'], desc="Converting tweets to vectors"):
    vector = tweet_to_vector(tweet)
    vectors.append(vector)

# Option 1: Store vectors in a new DataFrame column
df['tweet_vectors'] = vectors



Converting tweets to vectors: 100%|██████████| 8900/8900 [00:27<00:00, 322.70it/s]


In [ ]:
df['tweet_vectors']

0       [0.05283811, -0.08590957, 0.04215441, 0.017693...
1       [-0.0037022003, 0.0072014355, -0.02259598, 0.0...
2       [0.08475753, -0.07600954, -0.042283405, 0.0377...
3       [0.024447342, -0.05977322, -0.050316926, -0.02...
4       [-0.002654642, -0.07114219, -0.047089737, -0.0...
                              ...                        
8895    [-0.04786226, 0.0075260308, -0.012790812, -0.0...
8896    [-0.0267528, 0.00047889937, 0.039825298, -0.03...
8897    [-0.054867502, 0.004924008, 0.023037918, -0.04...
8898    [-0.05440314, -0.057027712, 0.017296702, -0.01...
8899    [0.056354567, 0.038590323, 0.022392513, -0.006...
Name: tweet_vectors, Length: 8900, dtype: object

In [ ]:
df.head()

,text,labels,tweet_vectors
0,Being s mom is cleaning 24/7 the same shit ove...,1,"[0.05283811, -0.08590957, 0.04215441, 0.017693..."
1,And now we have been given the walkthru book b...,0,"[-0.0037022003, 0.0072014355, -0.02259598, 0.0..."
2,Wishing YOU Peace Joy & Love! JoyTrain MentalH...,0,"[0.08475753, -0.07600954, -0.042283405, 0.0377..."
3,speak-no-evil monkey Can I Be Honest With You...,1,"[0.024447342, -0.05977322, -0.050316926, -0.02..."
4,Psy Do u hv any regrets? Me No Psy Are you hap...,0,"[-0.002654642, -0.07114219, -0.047089737, -0.0..."


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = pd.DataFrame(df['tweet_vectors'].tolist())
y = df['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=200, random_state=30)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')

0.8180495716844908

SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted')


0.8321685218267599